In [51]:
# These are the packages for completing this task. Not all of them are necessary but in case other scenario is required, these packages will be handy. 
import pandas as pd
import sys, getopt, pprint
import datetime 
import pymongo
from pymongo import MongoClient
import numpy as np
import pandas as pd
import MySQLdb
import sqlalchemy
import os
import bson
import re
from bson import json_util, ObjectId
import requests
import json, os
import configparser
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import warnings
import random
import pprint
from datetime import datetime
random.seed(datetime.now())

In [2]:
#Firstly, store following values for use of connect Jupyter Notebook with MySQL.
#The values are not necessarily same for everyone. One should install a proper version of MySQL Workbench and create a new connection. 
#This blog assumes that one has already created a usable MySQL database and tables are well structured. 

#Now enter the values for you database connection
dsn_database = "project"   # e.g. "MySQLdbtest"
dsn_hostname = "127.0.0.1" # e.g.: "mydbinstance.xyz.us-east-1.rds.amazonaws.com"
dsn_port = 3306            # e.g. 3306 
dsn_uid = "root"            # e.g. "user1"
dsn_pwd = "12345"           # e.g. "Password123"

In [3]:
# The python version I used is Python3 which needs you to use sqlalchemy connection instead of MySQLab to retrieve the data from MySQL database. 
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(dsn_uid, dsn_pwd, 
                                                      dsn_hostname, dsn_database))

In [4]:
#Now that we have the connection with MySQL, extract and store the table from the database.
#Pandas package is used for data cleaning to getting ready for MongoDB. 
tweets_df = pd.read_sql('SELECT * FROM tweets', con=database_connection)
game_df = pd.read_sql('SELECT * FROM game', con=database_connection)
publisher_df = pd.read_sql('SELECT * FROM publisher', con=database_connection)
tp_df = pd.read_sql('SELECT * FROM type1', con=database_connection)

In [5]:
#Data cleaning procedure begins, these steps should be created on your own since every one has different data and needs.
#Always make copies and check status of the dataframe in case the data cleaning went wrong and messed up useful data.
#If mistake is made, roll back to the step above and reload the data from MySQL database. 
tweets_df['Tweet_date']=tweets_df['Tweet_date'].astype('str')
tweets_df['Tweet_time']=tweets_df['Tweet_time'].astype('str')

In [6]:
#Cleaning cont.
tweets_df['Tweet_time']=tweets_df['Tweet_time'].str[7:-10]

In [7]:
#Cleaning cont.
tweets_df['Created_at']=tweets_df['Tweet_date']+' '+tweets_df['Tweet_time']
tweets_df=tweets_df.drop(['Tweet_date','Tweet_time'],axis=1)

In [8]:
#Cleaning cont.
tweets_df['Created_at']=pd.to_datetime(tweets_df['Created_at'])

In [9]:
#Make a final check point before migrating data into MongoDB since MongoDB is not convenient for data cleaning and making duplication and other mistakes are very easy in MongoDB.
tweets_df

,Favorites,Game_id,Hashtags,Location,Retweets,Screen_name,Tweet_text,Tweet_id1,Tweet_id2,total,Created_at
0,0,488,TrueAchievement,New York,0,RoninS00,I won the Bosom Buddies achievement in Monster...,985669091,305361408,0,2018-04-15 23:59:58
1,0,488,TrueAchievement,Luzern,0,mitzat,I won 4 achievements in Monster Hunter: World ...,985669090,852376576,0,2018-04-15 23:59:58
2,0,488,None,None,0,Taimi_wtc,Earned 2 PSN trophies (2 silver) in Monster Hu...,985668842,180530178,0,2018-04-15 23:58:59
3,0,488,None,None,0,dealPS4,Super bon plan : Pack PS4 Pro Blanche + Mons...,985668606,506647552,0,2018-04-15 23:58:03
4,3,488,None,Michigan,1,Menagese,Now live! Farming some Kushala for some horns ...,985668582,527860736,4,2018-04-15 23:57:57
5,0,488,None,None,4,MysticSpitfire,RT @Embozine: Streaming the good stuff aka Hun...,985668300,506988544,4,2018-04-15 23:56:50
6,0,488,None,"Elk Grove, CA",4,larncat,RT @Embozine: Streaming the good stuff aka Hun...,985668258,316468224,4,2018-04-15 23:56:40
7,0,488,None,"Seattle, WA",4,MHKogath,RT @Embozine: Streaming the good stuff aka Hun...,985668243,707723776,4,2018-04-15 23:56:36
8,0,488,PS4share,Castle Oblivion,0,Temp24601,Monster Hunter: World\n\xd8\xb5\xd9\x8a\xd9\x9...,985668227,358494720,0,2018-04-15 23:56:32
9,8,488,None,Dunder Mifflin,4,Embozine,Streaming the good stuff aka Hunter Monster Wo...,985668186,518335488,12,2018-04-15 23:56:23


In [10]:
#Next step is to create connection between MongoDB and Jupyter Notebook
#pymongo is the major package used for this task and almost all tasks on MongoDB.
#PLEASE follow the video in this link https://www.youtube.com/watch?v=_RQ4lET5ejw to setup your MongoDb as additional steps are required other than installing. 
#Similar to MySQL, MongoDB requires a independent connection. Although for MongoDB, the default connection on Windows OS should be the same.  
#To successfully connect with MongoDB, one needs to search "cmd" in Start menu and open a new "cmd" window, change directory to the installed MongoDB bin
# e.g. cd C:\Program Files\MongoDB\Server\3.6\bin
# Then connect the MongoDB with your required database workspace to start MongoDB connection. 
#e.g. mongod --dbpath D:\data\db
#Now the MongoDB ready for your control. 
#These information tells the server which client, which database, which collection is used. It automatically creates the database, collection for you when you insert data for the first time. 
client = MongoClient('localhost', 27017)
db = client['nosqlproject2']
collection=db['tweets']
tweets = db.tweets

In [11]:
#inser_many allows you to insert multiple objects into your Mongo database. In this example, each row of dtaframe equal to one object in MongoDB. 
#Instead of tables, rows and columns, MongoDB applies a concept of collection and object. One object has different status which refers to columns in MySQL database. 
tweets.insert_many(tweets_df.to_dict('records'))
#In this example, only one collection is needed which is the "tweets" table from MySQL database to answer questions. 

In [31]:
#1 What are people saying about me (somebody)?
#All tweets are collected based on keyword monster hunter world. All hashtags information is aggregated and counted to answer this question. 
for tt in tweets.aggregate([ {"$group": {"_id": "$Hashtags", "count": {"$sum": 1}}},{"$sort":{"count":-1}}  ]):
    pprint.pprint(tt)

{'_id': None, 'count': 350}
{'_id': 'MHWorld', 'count': 53}
{'_id': 'PS4live', 'count': 45}
{'_id': 'MonsterHunterWorld', 'count': 6}
{'_id': 'モンハン', 'count': 5}
{'_id': 'モンハンワールド', 'count': 5}
{'_id': 'PS4share', 'count': 5}
{'_id': 'TrueAchievement', 'count': 5}
{'_id': 'Störi', 'count': 3}
{'_id': 'Streaming', 'count': 3}
{'_id': 'Twitch', 'count': 3}
{'_id': 'monsterhunterworld', 'count': 3}
{'_id': 'MonsterHunterWorldSweepstakes', 'count': 3}
{'_id': 'GamesStruck4', 'count': 2}
{'_id': 'MONSTERHUNTERWORLDSwee', 'count': 2}
{'_id': 'SupportSmallStreamers', 'count': 2}
{'_id': 'PS4Pro', 'count': 2}
{'_id': 'deal', 'count': 2}
{'_id': 'exophase', 'count': 2}
{'_id': 'BonPlan', 'count': 2}
{'_id': 'games', 'count': 2}
{'_id': '2', 'count': 2}
{'_id': '1', 'count': 2}
{'_id': 'ps4', 'count': 2}
{'_id': 'Str', 'count': 2}
{'_id': 'etsy', 'count': 1}
{'_id': 'IAmACreator', 'count': 1}
{'_id': '02', 'count': 1}
{'_id': 'StreamAcademy', 'count': 1}
{'_id': 'twitchclips', 'count': 1}
{'_id'

In [42]:
#2 How viral are my posts?
# Retweets number should be enough to indicate how viral are each aggregated tweet text. 
for tt in db.tweets.aggregate([ {"$group" : {"_id":{"source":"$Tweet_text","status":"$Retweets"}, "count":{"$sum":1}}}, {"$sort":{"_id.status":-1}} ]):
    pprint.pprint(tt['_id'])

{'source': 'RT @Xbox: RT and follow for a chance to win a gigantic Monster '
           'Hunter: World prize pack. NoPurchNec. Ends 03/30/18. '
           '#MONSTERHUNTERWORLDSwee\\xe2\\x80\\xa6',
 'status': 20095}
{'source': 'RT @monsterhunter: \\xe2\\x9c\\xa8 New #MHWorld update coming your '
           'way: Kulve Taroth Siege. New limited-time game mode starting April '
           '19 \\xe2\\x9c\\xa8\\n\\n\\xe2\\x9c\\xa8 https://t.c\\xe2\\x80\\xa6',
 'status': 7447}
{'source': 'RT @4GamerNews: \\xe3\\x80\\x8cMONSTER HUNTER: '
           'WORLD\\xe3\\x80\\x8d\\xe7\\x84\\xa1\\xe6\\x96\\x99\\xe5\\xa4\\xa7\\xe5\\x9e\\x8b\\xe3\\x82\\xa2\\xe3\\x83\\x83\\xe3\\x83\\x97\\xe3\\x83\\x87\\xe3\\x83\\xbc\\xe3\\x83\\x88\\xe7\\xac\\xac2\\xe5\\xbc\\xbe\\xef\\xbc\\x8c4\\xe6\\x9c\\x8819\\xe6\\x97\\xa5\\xe9\\x85\\x8d\\xe4\\xbf\\xa1\\xe6\\xb1\\xba\\xe5\\xae\\x9a\\xe3\\x80\\x82\\xe8\\xbf\\xbd\\xe5\\x8a\\xa0\\xe3\\x83\\xa2\\xe3\\x83\\xb3\\xe3\\x82\\xb9\\xe3\\x82\\xbf\\xe3\\x83\\xbc\\xe3\\x81\\xaf\\xe3\\x

 'status': 0}
{'source': 'I went back and played a little of Monster Hunter Generations '
           'after living with World for the past few months and '
           'j\\xe2\\x80\\xa6 https://t.co/APwudBvbCt',
 'status': 0}
{'source': '@jakkujessica Monster Hunter World', 'status': 0}
{'source': 'Monster Hunter World Update 3.0 Brings A Host Of Quality-of-life '
           'Improvements Alongside Kulve Taroth Content - VG24\\xe2\\x80\\xa6 '
           'https://t.co/3rVvK3tZ1I',
 'status': 0}
{'source': 'fiquei desde as 4:30 da tarde tentando matar o nergigante no '
           'monster hunter world e s\\xc3\\xb3 fui conseguir agr pouco, bixo '
           'imundo do kct',
 'status': 0}
{'source': 'Check out my broadcast from my PlayStation 4! #PS4live (Monster '
           'Hunter: World)  live at https://t.co/zwU2rfQmq3',
 'status': 0}
{'source': 'Check out my broadcast from my PlayStation 4! #PS4live (Monster '
           'Hunter: World)  live at https://t.co/Yz89bdp7k4',
 'status': 0

In [77]:
#3 How much influence to my posts have?
# Retweets and favorites numbers are used to reflect the influence of a tweet text. 
#It is sorted by te total number of retweets and favorites means that it is assumed that there is no difference between one retweet and one favorite. 
for tt in db.tweets.aggregate([ {"$group" : {"_id":{"source":"$Tweet_text","status":["$Favorites","$Retweets"]}, "count":{"$sum":1}}}, {"$sort":{"_id.status":-1}} ]):
    pprint.pprint(tt['_id'])

{'source': 'RT @Xbox: RT and follow for a chance to win a gigantic Monster '
           'Hunter: World prize pack. NoPurchNec. Ends 03/30/18. '
           '#MONSTERHUNTERWORLDSwee\\xe2\\x80\\xa6',
 'status': [0, 20095]}
{'source': 'RT @monsterhunter: \\xe2\\x9c\\xa8 New #MHWorld update coming your '
           'way: Kulve Taroth Siege. New limited-time game mode starting April '
           '19 \\xe2\\x9c\\xa8\\n\\n\\xe2\\x9c\\xa8 https://t.c\\xe2\\x80\\xa6',
 'status': [0, 7447]}
{'source': 'RT @4GamerNews: \\xe3\\x80\\x8cMONSTER HUNTER: '
           'WORLD\\xe3\\x80\\x8d\\xe7\\x84\\xa1\\xe6\\x96\\x99\\xe5\\xa4\\xa7\\xe5\\x9e\\x8b\\xe3\\x82\\xa2\\xe3\\x83\\x83\\xe3\\x83\\x97\\xe3\\x83\\x87\\xe3\\x83\\xbc\\xe3\\x83\\x88\\xe7\\xac\\xac2\\xe5\\xbc\\xbe\\xef\\xbc\\x8c4\\xe6\\x9c\\x8819\\xe6\\x97\\xa5\\xe9\\x85\\x8d\\xe4\\xbf\\xa1\\xe6\\xb1\\xba\\xe5\\xae\\x9a\\xe3\\x80\\x82\\xe8\\xbf\\xbd\\xe5\\x8a\\xa0\\xe3\\x83\\xa2\\xe3\\x83\\xb3\\xe3\\x82\\xb9\\xe3\\x82\\xbf\\xe3\\x83\\xbc\\xe3\\x81\\x

 'status': [0, 0]}
{'source': "Vermintide 2 is the only game I've logged more hours on than "
           "Monster Hunter World so far this year. This game just doesn't get "
           'old.',
 'status': [0, 0]}
{'source': 'Me ha gustado un v\\xc3\\xaddeo de @YouTube '
           '(https://t.co/cndGW1FkJB - Let\\xc2\\xb4s Play Monster Hunter '
           'World 6 Godzilla \\xc2\\xbfEres T\\xc3\\xba?).',
 'status': [0, 0]}
{'source': "If you haven't played Monster Hunter World yet you should",
 'status': [0, 0]}
{'source': '[USA-GA][H] Ni No Kuni 2 Premium Edition, Monster Hunter World, '
           "Majora's Mask N3DSXL, PS1/2/3/4, GBA, Vita, and mor\\xe2\\x80\\xa6 "
           'https://t.co/9xCXQQEfjF',
 'status': [0, 0]}
{'source': 'Check out my broadcast from my PlayStation 4! #PS4live (Monster '
           'Hunter: World)  live at https://t.co/jQqAcJOc5N',
 'status': [0, 0]}
{'source': 'RETURN TO: Monster Hunter: WORLD - GOING AFTER DEVILJO '
           'https://t.co/Vyl8CPiQyS htt

In [85]:
#7 What topics are trending in my domain?
# The newest date of the tweets are 2018-04-19. The aggregation of all hashtags on this date is considered the trending topic in domain. 
d = datetime(2018,4,19,0)
for tt in tweets.aggregate([{"$match": {"Created_at":{"$gt":d}}},{"$group": {"_id":"$Hashtags","count":{"$sum":1}}},{"$sort":{"count":-1}}]):
    pprint.pprint(tt)

{'_id': None, 'count': 71}
{'_id': 'PS4live', 'count': 15}
{'_id': 'Störi', 'count': 3}
{'_id': 'Streaming', 'count': 3}
{'_id': 'Twitch', 'count': 3}
{'_id': 'Str', 'count': 2}
{'_id': 'GamesStruck4', 'count': 2}
{'_id': 'etsy', 'count': 1}
{'_id': 'IAmACreator', 'count': 1}
{'_id': 'SupportSmallerStreamers', 'count': 1}
{'_id': '02', 'count': 1}
{'_id': '14', 'count': 1}
{'_id': 'MHWorld', 'count': 1}
{'_id': 'twitchclips', 'count': 1}
{'_id': 'MONSTERHUNTERWORLDSwee', 'count': 1}
{'_id': 'monsterhunterworld', 'count': 1}
{'_id': 'StreamAcademy', 'count': 1}


In [ ]:
#MIT License

#Copyright <YEAR> <COPYRIGHT HOLDER>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.